In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf

In [ ]:
spy = yf.download('SPY', start='2025-01-01', end='2025-12-31', auto_adjust=False, group_by='tickers')['SPY']
data  =spy[['Open','High', 'Low', 'Adj Close']].copy()
data.tail()

[*********************100%***********************]  1 of 1 completed


Price,Open,High,Low,Adj Close
Date,,,,
2025-12-23,683.919983,688.200012,683.869995,687.960022
2025-12-24,687.950012,690.830017,687.799988,690.380005
2025-12-26,690.640015,691.659973,689.270020,690.309998
2025-12-29,687.539978,689.200012,686.070007,687.849976
2025-12-30,687.450012,688.559998,686.580017,687.010010


In [ ]:
data['return'] = data['Adj Close'].pct_change()
data.tail()

Price,Open,High,Low,Adj Close,return
Date,,,,,
2025-12-23,683.919983,688.200012,683.869995,687.960022,0.004570
2025-12-24,687.950012,690.830017,687.799988,690.380005,0.003518
2025-12-26,690.640015,691.659973,689.270020,690.309998,-0.000101
2025-12-29,687.539978,689.200012,686.070007,687.849976,-0.003564
2025-12-30,687.450012,688.559998,686.580017,687.010010,-0.001221


In [ ]:
data['state'] = data['return'].apply(lambda x: 'Up' if (x > 0.0015)
                                     else 'Down' if (x < -0.0015)
                                     else 'Flat')
data.tail()

Price,Open,High,Low,Adj Close,return,state,priorstate
Date,,,,,,,
2025-12-23,683.919983,688.200012,683.869995,687.960022,0.004570,Up,Up
2025-12-24,687.950012,690.830017,687.799988,690.380005,0.003518,Up,Up
2025-12-26,690.640015,691.659973,689.270020,690.309998,-0.000101,Flat,Up
2025-12-29,687.539978,689.200012,686.070007,687.849976,-0.003564,Down,Flat
2025-12-30,687.450012,688.559998,686.580017,687.010010,-0.001221,Flat,Down


In [ ]:
data['priorstate'] = data['state'].shift(1)
data.tail()

Price,Open,High,Low,Adj Close,return,state,priorstate
Date,,,,,,,
2025-12-23,683.919983,688.200012,683.869995,687.960022,0.004570,Up,Up
2025-12-24,687.950012,690.830017,687.799988,690.380005,0.003518,Up,Up
2025-12-26,690.640015,691.659973,689.270020,690.309998,-0.000101,Flat,Up
2025-12-29,687.539978,689.200012,686.070007,687.849976,-0.003564,Down,Flat
2025-12-30,687.450012,688.559998,686.580017,687.010010,-0.001221,Flat,Down


## Frequency Distribution Matrix

In [ ]:
states = data[['priorstate', 'state' ]].dropna()
states_mat = states.groupby(['priorstate', 'state']).size().unstack()
states_mat

state,Down,Flat,Up
priorstate,,,
Down,26,13,44
Flat,17,2,24
Up,40,28,54


In [ ]:
#Initial Transition Matrix
transition_matrix = states_mat.apply(lambda x: x/x.sum(), axis=1)
transition_matrix

state,Down,Flat,Up
priorstate,,,
Down,0.313253,0.156627,0.530120
Flat,0.395349,0.046512,0.558140
Up,0.327869,0.229508,0.442623


In [ ]:
t0 = transition_matrix.copy()
t1 = round(t0.dot(t0),4)
t1

state,Down,Flat,Up
priorstate,,,
Down,0.3339,0.1780,0.4881
Flat,0.3252,0.1922,0.4826
Up,0.3386,0.1636,0.4978


In [ ]:
t2 = round(t0.dot(t1),4)
t2

state,Down,Flat,Up
priorstate,,,
Down,0.3350,0.1726,0.4924
Flat,0.3361,0.1706,0.4933
Up,0.3340,0.1749,0.4911


In [ ]:
t3 = round(t0.dot(t2),4)
t3

state,Down,Flat,Up
priorstate,,,
Down,0.3346,0.1735,0.4919
Flat,0.3345,0.1738,0.4917
Up,0.3348,0.1732,0.4920


In [ ]:
#Alternative
pd.DataFrame(np.linalg.matrix_power(t0,4))

,0,1,2
0,0.334620,0.173508,0.491873
1,0.334471,0.173788,0.491741
2,0.334789,0.173164,0.492047


In [ ]:
#equilibrium matrix
i = 1
a = t0.copy()
b = t0.dot(t0)
while (not(a.equals(b))) :
  print(f"Iteration number: {i}")
  i += 1
  a = b.copy()
  b = b.dot(t0)